In [ ]:
import pandas as pd
import os
from pandas import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/sales_train.csv')
item_cat = pd.read_csv('/content/drive/MyDrive/Dataset/item_categories.csv')
items = pd.read_csv('/content/drive/MyDrive/Dataset/items.csv')
shops = pd.read_csv('/content/drive/MyDrive/Dataset/shops.csv')
sample = pd.read_csv('/content/drive/MyDrive/Dataset/sample_submission.csv')
test = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')

In [ ]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [ ]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [ ]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


Convert date to datetime format

In [ ]:
train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


creating pivot table

In [ ]:
dataset = train.pivot_table(index= ['shop_id','item_id'], values = ['item_cnt_day'], columns=['date_block_num'],fill_value=0, aggfunc='sum')
dataset.reset_index(inplace= True)
dataset

shop_id item_id item_cnt_day               ...                     
date_block_num                            0   1  2  3  4  ... 27 28 29 30 31 32 33
0                    0      30            0  31  0  0  0  ...  0  0  0  0  0  0  0
1                    0      31            0  11  0  0  0  ...  0  0  0  0  0  0  0
2                    0      32            6  10  0  0  0  ...  0  0  0  0  0  0  0
3                    0      33            3   3  0  0  0  ...  0  0  0  0  0  0  0
4                    0      35            1  14  0  0  0  ...  0  0  0  0  0  0  0
...                ...     ...          ...  .. .. .. ..  ... .. .. .. .. .. .. ..
424119              59   22154            1   0  0  0  0  ...  0  0  0  0  0  0  0
424120              59   22155            0   0  0  0  0  ...  0  0  0  0  0  0  0
424121              59   22162            0   0  0  0  0  ...  1  1  0  0  1  0  0
424122              59   22164            0   0  0  0  0  ...  2  0  0  1  0  0  0
424123              59   22167            0   0  0  0  0  ...  0  0  0  0  0  0  0

[424124 rows x 36 columns]

Data Processing

In [ ]:
test = test.drop(['ID'], axis=1)
test

,shop_id,item_id
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757
214198,45,19648


Merge pivot with test set

In [ ]:
dataset = pd.merge(test,dataset, on=['item_id','shop_id'], how='left')
dataset

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)","(item_cnt_day, 11)","(item_cnt_day, 12)","(item_cnt_day, 13)","(item_cnt_day, 14)","(item_cnt_day, 15)","(item_cnt_day, 16)","(item_cnt_day, 17)","(item_cnt_day, 18)","(item_cnt_day, 19)","(item_cnt_day, 20)","(item_cnt_day, 21)","(item_cnt_day, 22)","(item_cnt_day, 23)","(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,45,16188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,45,19648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Checking for null values

In [ ]:
dataset.isnull().sum()

shop_id                    0
item_id                    0
(item_cnt_day, 0)     102796
(item_cnt_day, 1)     102796
(item_cnt_day, 2)     102796
(item_cnt_day, 3)     102796
(item_cnt_day, 4)     102796
(item_cnt_day, 5)     102796
(item_cnt_day, 6)     102796
(item_cnt_day, 7)     102796
(item_cnt_day, 8)     102796
(item_cnt_day, 9)     102796
(item_cnt_day, 10)    102796
(item_cnt_day, 11)    102796
(item_cnt_day, 12)    102796
(item_cnt_day, 13)    102796
(item_cnt_day, 14)    102796
(item_cnt_day, 15)    102796
(item_cnt_day, 16)    102796
(item_cnt_day, 17)    102796
(item_cnt_day, 18)    102796
(item_cnt_day, 19)    102796
(item_cnt_day, 20)    102796
(item_cnt_day, 21)    102796
(item_cnt_day, 22)    102796
(item_cnt_day, 23)    102796
(item_cnt_day, 24)    102796
(item_cnt_day, 25)    102796
(item_cnt_day, 26)    102796
(item_cnt_day, 27)    102796
(item_cnt_day, 28)    102796
(item_cnt_day, 29)    102796
(item_cnt_day, 30)    102796
(item_cnt_day, 31)    102796
(item_cnt_day,

In [ ]:
dataset.fillna(0, inplace=True)
dataset.isnull().sum()

shop_id               0
item_id               0
(item_cnt_day, 0)     0
(item_cnt_day, 1)     0
(item_cnt_day, 2)     0
(item_cnt_day, 3)     0
(item_cnt_day, 4)     0
(item_cnt_day, 5)     0
(item_cnt_day, 6)     0
(item_cnt_day, 7)     0
(item_cnt_day, 8)     0
(item_cnt_day, 9)     0
(item_cnt_day, 10)    0
(item_cnt_day, 11)    0
(item_cnt_day, 12)    0
(item_cnt_day, 13)    0
(item_cnt_day, 14)    0
(item_cnt_day, 15)    0
(item_cnt_day, 16)    0
(item_cnt_day, 17)    0
(item_cnt_day, 18)    0
(item_cnt_day, 19)    0
(item_cnt_day, 20)    0
(item_cnt_day, 21)    0
(item_cnt_day, 22)    0
(item_cnt_day, 23)    0
(item_cnt_day, 24)    0
(item_cnt_day, 25)    0
(item_cnt_day, 26)    0
(item_cnt_day, 27)    0
(item_cnt_day, 28)    0
(item_cnt_day, 29)    0
(item_cnt_day, 30)    0
(item_cnt_day, 31)    0
(item_cnt_day, 32)    0
(item_cnt_day, 33)    0
dtype: int64

Drop shop_id and item_id

In [ ]:
dataset.drop(['shop_id','item_id'], inplace= True, axis=1)


In [ ]:
dataset

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)","(item_cnt_day, 11)","(item_cnt_day, 12)","(item_cnt_day, 13)","(item_cnt_day, 14)","(item_cnt_day, 15)","(item_cnt_day, 16)","(item_cnt_day, 17)","(item_cnt_day, 18)","(item_cnt_day, 19)","(item_cnt_day, 20)","(item_cnt_day, 21)","(item_cnt_day, 22)","(item_cnt_day, 23)","(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y= train['item_cnt_day']
x= train.drop(columns=['item_cnt_day'])
x.head()

,date,date_block_num,shop_id,item_id,item_price
0,2013-01-02,0,59,22154,999.00
1,2013-01-03,0,25,2552,899.00
2,2013-01-05,0,25,2552,899.00
3,2013-01-06,0,25,2554,1709.05
4,2013-01-15,0,25,2555,1099.00


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

2055094
880755


In [ ]:
y_train = dataset.iloc[:, -1:]
X_train = dataset.iloc[:, :-1]
X_test = dataset.iloc[:, 1:]

print(X_train.shape, y_train.shape, X_test.shape)

(214200, 33) (214200, 1) (214200, 33)


Defining the model

Using Linear Regression model

In [ ]:
import sklearn.metrics as sm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

#make predictions using linear regression
#price_predict =(linear_model.predict(X_test))




/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


LinearRegression()

In [ ]:
print('Linear Regression model score is:',linear_model.score(X_train,y_train))
print('MSE: ',mean_squared_error(y_train,linear_model.predict(X_train)))
linear_model.predict(X_test)


Linear Regression model score is: 0.5147429268717023
MSE:  14.952316144284213


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 1.86805549],
       [-0.05704087],
       [ 1.20825163],
       ...,
       [-0.47038433],
       [-0.05704087],
       [ 0.24268896]])

In [ ]:
y_pred = linear_model.predict(X_train)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 1.45046409],
       [-0.05704087],
       [ 2.62957075],
       ...,
       [ 1.57859293],
       [-0.05704087],
       [-0.55271714]])

In [ ]:
print(linear_model.score(X_train,y_train))

0.5147429268717023


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
pred = linear_model.predict(X_test)
pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 1.86805549],
       [-0.05704087],
       [ 1.20825163],
       ...,
       [-0.47038433],
       [-0.05704087],
       [ 0.24268896]])

Using RandomForest

In [ ]:
random_model = RandomForestRegressor(max_depth=5)
random_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(max_depth=5)

In [ ]:
print(linear_model.score(X_train,y_train))

0.5147429268717023


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
y_pred = linear_model.predict(X_train)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 1.45046409],
       [-0.05704087],
       [ 2.62957075],
       ...,
       [ 1.57859293],
       [-0.05704087],
       [-0.55271714]])

Using AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada_model = AdaBoostRegressor()
ada_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor()

In [ ]:
print(ada_model.score(X_train,y_train))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


0.8767149468785627


In [ ]:
y_pred = ada_model.predict(X_train)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([0.91534858, 1.64788238, 0.9329866 , ..., 1.64788238, 1.64788238,
       1.64788238])

In [ ]:
model = AdaBoostRegressor(base_estimator= RandomForestRegressor(max_depth=5), random_state=0, n_estimators=2000).fit(X_train,y_train) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred = model.predict(X_train)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([0.42264726, 0.19801706, 1.65812525, ..., 0.19801706, 0.19801706,
       0.19801706])

In [ ]:
print(model.score(X_train,y_train))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


0.9429033281380743


Prediction

In [ ]:
pred = model.predict(X_test)
pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([0.19801706, 0.19801706, 0.42264726, ..., 0.19801706, 0.19801706,
       0.19801706])

Submission 

In [ ]:
submission_file = pred
submission = pd.DataFrame({'ID':ID, 'item_cnt_month':submission_file.ravel()})
submission.to_csv = ('submission.csv', index= False)
submission.head()

SyntaxError: ignored